In [ ]:
try:
    import google.colab
    ! pip install summerepi2==1.1.1
    ! pip install estival
except:
    pass

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from jax import grad, jit, vmap, pmap
from jax import numpy as jnp

In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
import jax

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from   scipy.stats import norm

def hmc(logp, n_samples, x0, n_steps, step_size):
    """Run Hamiltonian Monte Carlo to draw `n_samples` from the log density
    `logp`, starting at initial state `x0`.
    """
    momenta_dist = norm(0, 1)

    # Kinetic and potential energy functions.
    T = lambda r: -momenta_dist.logpdf(r).sum()
    V = lambda x: -logp(x)

    V = jit(V)
    grad_V = jit(grad(V))
    dim = len(x0)
    samples = np.empty((n_samples, dim))
    samples[0] = x0
    
    accepted = np.empty(n_samples,dtype=bool)
    
    internal_steps = []

    for i in range(1, n_samples):

        x_curr = samples[i-1]
        r_curr = momenta_dist.rvs(size=dim)
        x_prop, r_prop, interim = leapfrog(x_curr, r_curr, n_steps, step_size, grad_V)

        H_prop = T(r_prop) + V(x_prop)
        H_curr = T(r_curr) + V(x_curr)
        alpha  = np.exp(-H_prop + H_curr)

        if np.random.uniform(0, 1) < alpha:
            x_curr = x_prop
            accepted[i] = True
        else:
            accepted[i] = False
            
        samples[i] = x_curr
        internal_steps.append(interim)
        

    return samples, accepted, internal_steps


def leapfrog(x, r, n_steps, step_size, grad_V):
    """Run the leapfrog integrator forward `n_steps` using step size
    `step_size`.
    """
    x, r = x[:], r[:]
    interim = []
    for _ in range(n_steps):
        r = r - (step_size / 2) * grad_V(x)
        x = x + step_size * r
        r = r - (step_size / 2) * grad_V(x)
        interim.append(x)
    r = -r
    return x, r, np.array(interim)


def log_rosen(x):
    """Compute the log of the Rosenbrock density.
    """
    return -((1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2) / 20


samples, accepted, internal = hmc(
    logp=log_rosen,
    n_samples=1000,
    x0=np.random.uniform(low=[-3, -3], high=[3, 10], size=2),
    n_steps=20,
    step_size=0.03
)

plt.plot(samples[:, 0], samples[:, 1])
plt.show()


In [ ]:
step = 50
int_step_vals = internal[step]
plt.plot(int_step_vals[:, 0], int_step_vals[:, 1])
plt.show()
samples[step]

In [ ]:
chain_res = []
for chain in range(4):
    samples,_,_ = hmc(
        logp=log_rosen,
        n_samples=1000,
        x0=np.random.uniform(low=[-3, -3], high=[3, 10], size=2),
        n_steps=20,
        step_size=0.03
    )
    chain_res.append(samples)


In [ ]:
for chain in range(4):
    cur_chain = chain_res[chain]
    plt.plot(cur_chain[:, 0], cur_chain[:, 1])

plt.show()


In [ ]:
comp_model = CompartmentalModel([0.0,100.0],["S","I","R"],["I"])
comp_model.set_initial_population({"S": 100.0, "I": 10.0})
comp_model.add_infection_frequency_flow("infection",Parameter("contact_rate"),"S","I")
comp_model.add_transition_flow("recovery",Parameter("recovery_rate"),"I","R")

In [ ]:
parameters = {
    "contact_rate": 0.5,
    "recovery_rate": 0.1
}

In [ ]:
comp_model.run(parameters)

In [ ]:
res = comp_model.get_outputs_df()

In [ ]:
from estival.targets import NegativeBinomialTarget

In [ ]:
t = NegativeBinomialTarget("infections", res["I"], 5.0)

In [ ]:
te = t.get_evaluator(res.index)

In [ ]:
te.evaluate(np.array(res["I"]),parameters)

In [ ]:
mrunner = comp_model.get_runner(parameters)#, dyn_params=list(parameters))

In [ ]:
def model_logp(parameters):
    res = mrunner._run_func(parameters)
    logp = te._eval_func(res["outputs"][:,1],parameters)
    return logp

In [ ]:
parameters

In [ ]:
grad_logp = jit(grad(model_logp))

In [ ]:
grad_logp(parameters)

In [ ]:
contact_rate_samples = np.linspace(0.1,0.8,1000)

In [ ]:
cr_grads = {
    "contact_rate": [],
    "recovery_rate": []
}
for cr in contact_rate_samples:
    cur_p = parameters.copy()
    cur_p.update({"contact_rate": cr})
    cur_grad = grad_logp(cur_p)
    for k, v in cur_grad.items():
        cr_grads[k].append(float(v))
        
pd.DataFrame(cr_grads,index=contact_rate_samples).plot()

In [ ]:
def model_logp_float(x):
    parameters = {
        "contact_rate": x[0],
        "recovery_rate": x[1]
    }
    res = mrunner._run_func(parameters)
    logp = te._eval_func(res["outputs"][:,1],parameters)
    return logp

In [ ]:
samples, accepted, internal = hmc(
    logp=model_logp_float,
    n_samples=1000,
    x0=np.random.uniform(low=[0.1, 0.01], high=[1.0, 1.0], size=2),
    n_steps=20,
    step_size=0.03
)

In [ ]:
plt.plot(samples[:, 0], samples[:, 1])
plt.show()
